# Use Case 1: NIWA Climate Projections Analysis

Climate resilience planning requires analysing vast temporal and spatial datasets. Using NIWA's 2024 climate projections, we'll visualise how **average daily maximum air temperatures** are projected to change across New Zealand, comparing future scenarios against historical baselines.

## Key Technical Highlights

- **Dataset Scale:** NIWA 2024 climate projections covering New Zealand
- **Performance:** GPU-accelerated rendering of temperature gradients
- **Interactivity:** Real-time exploration of climate change impacts
- **Memory Efficiency:** PyArrow's zero-copy operations for large time series

## What You'll See

This demonstration showcases how properties and regions across Aotearoa New Zealand will experience changing temperature patterns, providing actionable insights for climate adaptation planning through high-performance geospatial visualisation.

## Datasets

### Climate Projections: NIWA 2024

Our climate data comes from NIWA's latest 2024 climate projections, built on the **Shared Socioeconomic Pathways (SSPs)** framework from the IPCC's Sixth Assessment Report.
<img src="../images/niwa_maps.png" alt="drawing" width="1200"/>

#### Shared Socioeconomic Pathways (SSPs)

SSPs represent different trajectories of societal development and their implications for greenhouse gas emissions:

- **SSP2-4.5 ("Middle of the Road")**: Moderate emissions scenario with gradual decoupling of economic growth from emissions. Represents a world following historical patterns with some progress toward sustainability goals. **It assumes that warming reaches 2.7˚C by 2100.**

- **SSP5-8.5 ("Fossil-fueled Development")**: High emissions scenario characterised by rapid economic growth powered by fossil fuels. Represents continued reliance on carbon-intensive energy sources with limited climate policy intervention. **It assumes warming of more than 4˚C by 2100.**

For this analysis, we focus on **average daily maximum air temperature** projections across New Zealand, comparing these future scenarios against the 1986-2015 historical baseline period.

<img src="../images/CS2023_2638_Graph-Carbon-dioxide__FocusFillWyIwLjAwIiwiMC4wMCIsNzk1LDY2NV0.jpg" alt="drawing" width="600"/>

### Property Boundaries: LINZ Unit of Property Layer

Our spatial analysis framework uses **Land Information New Zealand's (LINZ) new Unit of Property layer** - a comprehensive national dataset providing:

- **Coverage**: All property boundaries across New Zealand
- **Geometry**: High-precision polygon boundaries for individual properties
- **Attributes**: Property identifiers, areas, and administrative classifications
- **Currency**: Regularly updated to reflect current cadastral information

This dataset enables property-level climate impact assessment, allowing us to understand how temperature changes will affect individual properties and local communities across the country.


In [ ]:
from ipywidgets import FloatRangeSlider, jsdlink
from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
from lonboard.layer_extension import DataFilterExtension
from palettable.colorbrewer.sequential import Oranges_8
from pyarrow.compute import divide, drop_null
from pyarrow.compute import max as pc_max
from pyarrow.compute import min as pc_min
from pyarrow.parquet import read_table
from sidecar import Sidecar

In [ ]:
# Get data
climate_results = read_table("../data/climate_results.parquet")
climate_results = drop_null(climate_results)

In [ ]:
# Set how we want to be able to filter the map
filter_extension = DataFilterExtension(filter_size=1)
filter_values = climate_results["Historical"]
initial_filter_range = [float(pc_min(filter_values)), float(pc_max(filter_values))]

In [ ]:
# Set how we want to colour the map
values = climate_results["SSP5-RCP8.5 (2040-2060)"]
normalised_future_values = values.to_numpy() / float(pc_max(values))
colour_map = apply_continuous_cmap(normalised_future_values, Oranges_8)

In [ ]:
# Create a layer
layer = ScatterplotLayer(
    climate_results,
    get_fill_color=colour_map,
    radius_min_pixels=2,
    radius_max_pixels=20,
    get_filter_value=filter_values,
    filter_range=initial_filter_range,
    extensions=[filter_extension],
    pickable=True,
)

In [ ]:
# And add the layer to a map
map_ = Map(
    layer,
    show_tooltip=True,
    show_side_panel=True,
    view_state={"latitude": -41.286062, "longitude": 172.760010, "zoom": 4.1},
)
sc = Sidecar(title="Climate")
with sc:
    display(map_)

In [ ]:
# Create a slider to find data that matches the historical
slider = FloatRangeSlider(
    value=initial_filter_range,
    min=initial_filter_range[0],
    max=initial_filter_range[1],
    step=1,
    description="Historical: ",
)
slider

In [ ]:
_ = jsdlink((slider, "value"), (layer, "filter_range"))